# A very simple example of star/galaxy separation

### First set up some `matplotlib` defaults

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (13, 8)

### Next import modules for use later

In [ ]:
import numpy
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Point2I, Extent2I, Box2D

### Define a utility function for use later

This function is a very simple image display tool.  It will plot a sub-region if you want and will overlay sources if you pass a catalog.

In [ ]:
def overlay_masks(exposure, bbox=None, sources=None):
    masked_image = exposure.getMaskedImage()
    if bbox is not None:
        masked_image = masked_image.Factory(masked_image, bbox)
    (img, mask, var) = masked_image.getArrays()
    plt.imshow(numpy.log10(img-2*img.min()), interpolation='nearest', cmap='gray')
    plt.imshow(mask, interpolation='nearest', cmap='Blues', alpha=0.3)
    if sources is not None:
        testbox = Box2D(bbox) # convert to a floating point box
        for source in sources:
            if testbox.contains(source.getCentroid()):
                # Need to account for difference in pixel indexing between matplotlib and stack
                plt.scatter(source.getX()-testbox.getMinX()-0.5, source.getY()-testbox.getMinY()-0.5, 
                            marker='+', c='r', alpha=0.5)
    plt.show()

### Make a butler
The `Butler` is the data abstraction layer for our stack.  Simply instantiate with a path to a valid repository.  Then you can ask for data be type and a set of unique identifiers.  In this case, I'm getting a calibrated exposure, or `calexp`.

In [ ]:
butler = Butler('/project/shared/data/RSP_CHECK_REPO/butler.yaml')
exp = butler.get('calexp', visit=1228, detector=87, instrument='HSC', collections='HSC/defaults')

### Now let's do something with the exposure
First we construct a bounding box for a subregion in the image.  Then get a source catalog using the `Butler`.  Finally, use the utility function above to plot the image, masks, and source.

In [ ]:
bbox = Box2I(Point2I(1350, 1200), Extent2I(400,400))
sources = butler.get('src', visit=1228, detector=87, instrument='HSC', collections='HSC/defaults')
overlay_masks(exp, bbox=bbox, sources=sources)

### Let's do a simple analysis
First calculate the ratio of the model flux to the PSF flux.  This is basically a measure of how much the source looks like a point source.

In [ ]:
x = sources.getPsfInstFlux()
y = sources.getModelInstFlux()/sources.getPsfInstFlux()
x = numpy.array(x)
y = numpy.array(y)

The stack provides a measurement for extendedness as well.  Separate stars from galaxies using this value.

In [ ]:
extend = sources.get('base_ClassificationExtendedness_value')
stars = numpy.where(extend==0.0)[0]
galaxies = numpy.where(extend>0.)[0]
plt.scatter(x[stars], y[stars], alpha=0.3)
plt.scatter(x[galaxies], y[galaxies], c='r', alpha=0.3)
plt.xlim(100000, 1000)
plt.ylim(-1, 5)
plt.show()

Notice how all points above a certain threshold are red.  This simply shows that the default star/galaxy separator in this reduction is a cut in the model flux/PSF flux ratio.